# Importing the Libraries

In [ ]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime

# Loading the data

In [ ]:
data = pd.read_csv('Daily Household Transactions.csv')

In [ ]:
data.head()

In [ ]:
data.shape

# Data-Preprocessing

In [ ]:
def ceil_to_nearest_tens(amount):
    return math.ceil(amount / 10) * 10

# Apply the function to the 'Amount' column
data['Rounded_to_Tens'] = data['Amount'].apply(ceil_to_nearest_tens)

In [ ]:
data.head()

In [ ]:
data['Extra_from_Exp'] = data['Rounded_to_Tens'] - data['Amount']

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.isnull().sum()

In [ ]:
data['Subcategory'].fillna('NA', inplace = True)
data['Note'].fillna('NA', inplace = True)

In [ ]:
data.isnull().sum()

In [ ]:
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True, errors='coerce')

data['Date_only'] = data['Date'].dt.date

# Group by the date and sum the 'Extra_from_Exp'
result = data.groupby('Date_only')['Extra_from_Exp'].sum().reset_index()

# Rename the columns for clarity
result.columns = ['Date', 'Total_Extra_from_Exp_Daywise']

result

In [ ]:
df = pd.DataFrame(result)
df

# Integrating the API

In [ ]:
# Define a list of stock symbols to analyze
stocks = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA']  # Example list, replace with your desired symbols

# Create a DataFrame to store CAGR values for each stock
cagr_data = {'Symbol': [], 'CAGR': []}

# Loop through each stock symbol
for symbol in stocks:
    # Download historical data
    data = yf.download(symbol, start='2017-01-01', end='2018-09-20', progress=False)
    
    # Calculate CAGR
    start_price = data['Adj Close'].iloc[0]
    end_price = data['Adj Close'].iloc[-1]
    num_years = len(data) / 252  # Assuming 252 trading days in a year
    cagr = ((end_price / start_price) ** (1 / num_years)) - 1
    
    # Append symbol and CAGR to DataFrame
    cagr_data['Symbol'].append(symbol)
    cagr_data['CAGR'].append(cagr)

# Convert the dictionary to a DataFrame
cagr_df = pd.DataFrame(cagr_data)

# Round off the CAGR values to two decimal places
cagr_df['CAGR'] = cagr_df['CAGR'].round(2)

# Sort DataFrame by CAGR in descending order
cagr_df = cagr_df.sort_values(by='CAGR', ascending=False)

# Display the top stocks with the highest CAGR
print("Top stocks with the highest CAGR:")
cagr_df

In [ ]:
# Convert the 'Date' column to datetime format and handle errors
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Drop rows where 'Date' could not be parsed
df = df.dropna(subset=['Date'])

# Fetch historical stock price data for a symbol
symbol = 'AMZN'  # Example stock symbol, replace with your SIP or stock symbol
stock_data = yf.download(symbol, start='2015-01-13', end='2018-09-20', progress=False)

# Ensure the stock_data index is in datetime format
stock_data.index = pd.to_datetime(stock_data.index)

# Reset the index to make Date a column in stock_data and rename it
stock_data.reset_index(inplace=True)
stock_data.rename(columns={'Date': 'Date_Stock'}, inplace=True)

# Remove time component from 'Date' columns
df['Date'] = df['Date'].dt.floor('D')
stock_data['Date_Stock'] = stock_data['Date_Stock'].dt.floor('D')

# Merge on the date columns
merged_data = pd.merge(df, stock_data, left_on='Date', right_on='Date_Stock', how='left', suffixes=('_exp', '_stock'))

# Display the merged data
merged_data

In [ ]:
# Fill NaN values in numerical columns with the mean of each column
merged_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']] = merged_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].fillna(merged_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].mean())

# Forward fill NaN values in other columns
merged_data.fillna(method='ffill', inplace=True)

# Drop rows with NaT values in 'Date_Stock' column
merged_data = merged_data.dropna(subset=['Date_Stock'])

# Print the cleaned DataFrame
print(merged_data)

In [ ]:
dataframe = pd.DataFrame(merged_data)
dataframe

In [ ]:
def calculate_sip_returns(df):
    df['Investment'] = df['Total_Extra_from_Exp_Daywise']
    df['Cumulative_Investment'] = df['Investment'].cumsum()
    
    # Calculate daily returns based on adjusted closing price
    df['Daily_Returns'] = df['Adj Close'].pct_change().fillna(0)
    
    # Calculate cumulative returns
    df['Cumulative_Returns'] = (df['Daily_Returns'] + 1).cumprod() * df['Cumulative_Investment']
    
    return df

# Assuming merged_data is your DataFrame
returns = calculate_sip_returns(merged_data)

returns = returns.reset_index(drop=True)
returns.index += 1

In [ ]:
# Printing the Investment and Returns
returns[['Investment', 'Date', 'Cumulative_Investment', 'Daily_Returns', 'Cumulative_Returns']]

In [ ]:
# Calculate total amount invested
total_invested = merged_data['Cumulative_Investment'].iloc[-1]

# Calculate total returns
total_returns = round(merged_data['Cumulative_Returns'].iloc[-1], 2)

# Calculate difference between total returns and total invested
difference = round(total_returns - total_invested, 2)

# Calculate percentage returns
percentage_returns = round((difference / total_invested) * 100, 2)

print("Total amount invested:", total_invested)
print("Total returns:", total_returns)
print("Difference:", difference)
print("Percentage returns:", percentage_returns, "%")

In [ ]:
# Plot the results
plt.figure(figsize=(14, 7))
plt.plot(returns.index, returns['Cumulative_Investment'], label='Cumulative Investment')
plt.plot(returns.index, returns['Cumulative_Returns'], label='Cumulative Returns')
plt.xlabel('Days')
plt.ylabel('Amount')
plt.title('SIP Returns vs. Cumulative Investment')
plt.legend()
plt.show()

In [ ]:
num_years = (returns['Date'].max() - returns['Date'].min()).days / 365.25
annualized_return = (total_returns / total_invested) ** (1 / num_years) - 1
print(f"Annualized Return: {annualized_return:.2%}")

In [ ]:
monthly_returns = returns.resample('M', on='Date').last()
monthly_returns['Monthly_Investment'] = returns.resample('M', on='Date')['Investment'].sum().cumsum()
monthly_returns[['Monthly_Investment', 'Cumulative_Returns']].plot(figsize=(10, 6))
plt.title('Monthly SIP Investment and Returns')
plt.xlabel('Date')
plt.ylabel('Amount (INR)')
plt.grid(True)
plt.show()

In [ ]:
daily_volatility = returns['Daily_Returns'].std()
annual_volatility = daily_volatility * math.sqrt(252)
print(f"Annual Volatility: {annual_volatility:.2%}")


In [ ]:
market_index = yf.download('^NSEI', start='2015-01-13', end='2018-09-20', progress=False)  # NIFTY 50
market_index['Cumulative_Market_Returns'] = (market_index['Adj Close'].pct_change().fillna(0) + 1).cumprod()
market_index = market_index[['Adj Close', 'Cumulative_Market_Returns']].rename(columns={'Adj Close': 'Market_Close'})

merged_data = merged_data.merge(market_index, left_on='Date', right_index=True, how='left')

plt.figure(figsize=(10, 6))
plt.plot(merged_data['Date'], merged_data['Cumulative_Returns'], label='SIP Returns')
plt.plot(merged_data['Date'], merged_data['Cumulative_Market_Returns'] * total_invested, label='Market Returns')
plt.xlabel('Date')
plt.ylabel('Amount (INR)')
plt.title('SIP Returns vs. Market Returns')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def simulate_sip(amount):
    returns['Investment'] = returns['Total_Extra_from_Exp_Daywise'] * amount
    returns['Cumulative_Investment'] = returns['Investment'].cumsum()
    returns['Cumulative_Returns'] = (returns['Daily_Returns'] + 1).cumprod() * returns['Cumulative_Investment']
    return returns

# Example: Simulate with different SIP amounts
for amount in [0.5, 1, 2]:  # 50%, 100%, and 200% of the original SIP amount
    simulation = simulate_sip(amount)
    plt.plot(simulation['Date'], simulation['Cumulative_Returns'], label=f'SIP Amount x{amount}')

plt.xlabel('Date')
plt.ylabel('Amount (INR)')
plt.title('SIP Returns with Different SIP Amounts')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import plotly.express as px

fig = px.line(returns, x='Date', y=['Cumulative_Investment', 'Cumulative_Returns'], labels={'value': 'Amount (INR)'}, title='Projected SIP Returns from Daily Extra Expenditure (Real-Time)')
fig.show()